In [64]:
import warnings

import numpy as np
import pandas as pd

from jre_utils.datapath import (
    factor_data_paths,
    model_built_data_paths,
    get_derived_csv_path,
    get_derived_lpa_path,
    get_derived_plps_path,
)

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [65]:
asset_type = "condo"
years_ahead = 2

metrics = {
    "weighted_mean": "unit_price_wmean",
    "weighted_median": "unit_price_wmedian",
    "mean": "unit_price_mean",
    "median": "unit_price_median",
    "weighted_mean_smoothed": "unit_price_wmean_smoothed",
    "weighted_median_smoothed": "unit_price_wmedian_smoothed",
    "mean_smoothed": "unit_price_mean_smoothed",
    "median_smoothed": "unit_price_median_smoothed",
}

dataset_paths = {
    "transactions": get_derived_csv_path(asset_type),
    "lpa": get_derived_lpa_path(),
    "plps": get_derived_plps_path()
}

granularity_columns = ["area", "area_code"]
group_by_columns = granularity_columns + ["year"]
display_columns = ["unit_price", "total_traded_area", "count"]

# metric_key_unsmoothed = "weighted_median"
metric_key_unsmoothed = "median"
metric_unsmoothed = metrics[metric_key_unsmoothed]

metric_key = f"{metric_key_unsmoothed}_smoothed"
metric = metrics[metric_key]

metric_pct_chg = metric + "_pct_chg"
upcoming_metric = "upcoming_" + metric


In [66]:
dataset_key = "transactions"
core_path = dataset_paths[dataset_key]
population_path = factor_data_paths["processed"]["population"]["municipality"]
migration_path = factor_data_paths["processed"]["migration"]["municipality"]
taxable_income_path = factor_data_paths["processed"]["taxable_income"]["municipality"]
new_dwellings_path = factor_data_paths["processed"]["new_dwellings"]["municipality"]
lfs_revenue_path = factor_data_paths["processed"]["lfs_revenue_breakdown"]["municipality"]

dataset_name = f"sequence_{dataset_key}_{asset_type}_{metric_key}_{years_ahead}"
model_built_data_path = model_built_data_paths[dataset_name]

In [67]:
df = pd.read_csv(core_path)
df = df.assign(asset_type=asset_type)

population_df = pd.read_csv(population_path)
migration_df = pd.read_csv(migration_path)
taxable_income_df = pd.read_csv(taxable_income_path)
new_dwellings_df = pd.read_csv(new_dwellings_path)
lfs_revenue_df= pd.read_csv(lfs_revenue_path)

df = (
    df.merge(population_df, on=group_by_columns, how="left")
    .merge(migration_df, on=group_by_columns, how="left")
    .merge(taxable_income_df, on=group_by_columns, how="left")
    .merge(new_dwellings_df, on=group_by_columns, how="left")
    .merge(lfs_revenue_df, on=group_by_columns, how="left")
)

In [68]:
# prepare metrics
df = df.sort_values(by=group_by_columns, ascending=True)
df[metric_pct_chg] = df.groupby(granularity_columns)[metric].pct_change(periods=years_ahead)
# df = df[~df[metric_pct_chg].isna()] # I don't want to drop the data from the first two years. I.e. 2007 should have 2005 and 2006 data in it's window

In [69]:
# prepare additional factors
df["count_growth"] = df.groupby(granularity_columns)["count"].pct_change()
df["yearly_price_growth"] = df.groupby(granularity_columns)[metric].pct_change()

for column in ["count", "total_traded_area", metric]:
    df[f"{column}_log"] = df[column].apply(lambda x: np.log10(1 + x))
    df[f"{column}_log_normalized_yearly"] = df.groupby("year")[f"{column}_log"].transform(
        lambda x: (x - x.mean()) / x.std()
    )

for column in ["count_growth", "yearly_price_growth", metric_pct_chg]: # metric_pct_chg_normalized_yearly will be the key metric
    df[f"{column}_normalized_yearly"] = df.groupby("year")[column].transform(
        lambda x: (x - x.mean()) / x.std()
    )

In [70]:
df["migrations_is_available"] = df["net_migration_ratio"].notnull().astype(int)
df["taxable_income_is_available"] = df["taxable_income"].notnull().astype(int)
df["total_tax_is_available"] = df["total_tax"].notnull().astype(int)
df["dwellings_is_available"] = df["new_dwellings"].notnull().astype(int)
df["metric_pct_chg_is_available"] = df[metric_pct_chg].notnull().astype(int)

In [71]:
log_normalize_columns = [
    metric,
    "count",
    "total_traded_area",
    "in_migrations",
    "out_migrations",
    "population",
    "taxpayer_count",
    "taxable_income",
    "taxable_income_per_taxpayer",
    "total_tax",
    "new_dwellings",
    "existing_dwellings",
]

normalize_columns = [
    metric_pct_chg,
    "count_growth",
    "yearly_price_growth",
    "total_tax_growth",
    "taxable_income_growth",
    "taxable_income_per_taxpayer_growth",
    "net_migration_ratio",
    "new_dwellings_ratio",
    "taxpayer_count_growth",
]

maintain_columns = [
    "migrations_is_available",
    "taxable_income_is_available",
    "dwellings_is_available",
    "total_tax_is_available",
    "metric_pct_chg_is_available"
]

id_columns = ["area_code", "area", "year", "asset_type"]

feature_columns = (
    [f"{column}_log_normalized_yearly" for column in log_normalize_columns]
    + [f"{column}_normalized_yearly" for column in normalize_columns]
    + maintain_columns
)

final_columns = id_columns + normalize_columns + log_normalize_columns + feature_columns

# add ratios and growths if necessary

df = df[final_columns]

In [72]:
df.to_csv(model_built_data_path, index=False)

In [73]:
df[df["unit_price_median_smoothed_pct_chg"] < -1]

,area_code,area,year,asset_type,unit_price_median_smoothed_pct_chg,count_growth,yearly_price_growth,total_tax_growth,taxable_income_growth,taxable_income_per_taxpayer_growth,net_migration_ratio,new_dwellings_ratio,taxpayer_count_growth,unit_price_median_smoothed,count,total_traded_area,in_migrations,out_migrations,population,taxpayer_count,taxable_income,taxable_income_per_taxpayer,total_tax,new_dwellings,existing_dwellings,unit_price_median_smoothed_log_normalized_yearly,count_log_normalized_yearly,total_traded_area_log_normalized_yearly,in_migrations_log_normalized_yearly,out_migrations_log_normalized_yearly,population_log_normalized_yearly,taxpayer_count_log_normalized_yearly,taxable_income_log_normalized_yearly,taxable_income_per_taxpayer_log_normalized_yearly,total_tax_log_normalized_yearly,new_dwellings_log_normalized_yearly,existing_dwellings_log_normalized_yearly,unit_price_median_smoothed_pct_chg_normalized_yearly,count_growth_normalized_yearly,yearly_price_growth_normalized_yearly,total_tax_growth_normalized_yearly,taxable_income_growth_normalized_yearly,taxable_income_per_taxpayer_growth_normalized_yearly,net_migration_ratio_normalized_yearly,new_dwellings_ratio_normalized_yearly,taxpayer_count_growth_normalized_yearly,migrations_is_available,taxable_income_is_available,dwellings_is_available,total_tax_is_available,metric_pct_chg_is_available


In [74]:
df.describe()

,area_code,year,unit_price_median_smoothed_pct_chg,count_growth,yearly_price_growth,total_tax_growth,taxable_income_growth,taxable_income_per_taxpayer_growth,net_migration_ratio,new_dwellings_ratio,taxpayer_count_growth,unit_price_median_smoothed,count,total_traded_area,in_migrations,out_migrations,population,taxpayer_count,taxable_income,taxable_income_per_taxpayer,total_tax,new_dwellings,existing_dwellings,unit_price_median_smoothed_log_normalized_yearly,count_log_normalized_yearly,total_traded_area_log_normalized_yearly,in_migrations_log_normalized_yearly,out_migrations_log_normalized_yearly,population_log_normalized_yearly,taxpayer_count_log_normalized_yearly,taxable_income_log_normalized_yearly,taxable_income_per_taxpayer_log_normalized_yearly,total_tax_log_normalized_yearly,new_dwellings_log_normalized_yearly,existing_dwellings_log_normalized_yearly,unit_price_median_smoothed_pct_chg_normalized_yearly,count_growth_normalized_yearly,yearly_price_growth_normalized_yearly,total_tax_growth_normalized_yearly,taxable_income_growth_normalized_yearly,taxable_income_per_taxpayer_growth_normalized_yearly,net_migration_ratio_normalized_yearly,new_dwellings_ratio_normalized_yearly,taxpayer_count_growth_normalized_yearly,migrations_is_available,taxable_income_is_available,dwellings_is_available,total_tax_is_available,metric_pct_chg_is_available
count,5111.000000,5111.000000,4439.000000,4770.000000,4770.000000,5078.000000,4798.000000,4798.000000,5111.000000,4543.000000,4798.000000,5.111000e+03,5111.000000,5111.000000,5111.000000,5111.000000,5.111000e+03,4.798000e+03,4.798000e+03,4798.000000,5.088000e+03,4543.000000,4.543000e+03,5.111000e+03,5.111000e+03,5.111000e+03,5111.000000,5111.000000,5111.000000,4798.000000,4798.000000,4798.000000,5088.000000,4543.000000,4543.000000,4.439000e+03,4.770000e+03,4.770000e+03,5078.000000,4798.000000,4798.000000,5111.000000,4543.000000,4798.000000,5111.0,5111.000000,5111.000000,5111.000000,5111.000000
mean,19947.395422,2014.491489,0.034422,0.082175,0.015699,0.008169,0.007731,0.000185,0.000387,0.016366,0.007442,2.624998e+05,154.772452,9009.187048,12351.367247,12093.226163,2.806702e+05,1.274699e+05,4.541998e+08,3476.133216,4.540604e+07,2360.975787,1.393530e+05,3.114098e-16,-6.673068e-17,2.224356e-16,1.241783,1.238111,1.224426,1.356171,1.403400,1.263926,1.324700,0.804200,0.853338,-3.201364e-18,3.724019e-18,1.191686e-17,0.079565,0.130657,-0.048631,0.482910,0.328497,0.344627,1.0,0.938760,0.888867,0.995500,0.868519
std,10154.971618,4.618083,0.114343,0.395240,0.067213,0.039129,0.031488,0.022677,0.005111,0.006223,0.016481,1.758514e+05,351.552632,19571.514183,21145.987577,20279.820890,3.811617e+05,1.735894e+05,6.619150e+08,797.229365,7.774505e+07,3633.415767,1.941294e+05,9.983352e-01,9.983352e-01,9.983352e-01,0.591924,0.611579,0.570563,0.572373,0.562609,1.096725,0.509559,0.769932,0.920230,9.983086e-01,9.983211e-01,9.983211e-01,0.770839,0.511619,0.537131,0.577406,0.822003,0.505988,0.0,0.239794,0.314327,0.066938,0.337959
min,1100.000000,2005.000000,-0.594214,-0.812500,-0.461342,-0.466623,-0.313802,-0.192218,-0.036352,0.001265,-0.337342,6.318009e+04,3.000000,100.000000,292.000000,443.000000,1.628800e+04,7.118000e+03,2.125598e+07,2441.662747,2.089891e+06,44.000000,1.578900e+04,-2.527400e+00,-2.078430e+00,-2.820555e+00,-0.669647,-0.485729,-0.308341,-0.211530,-0.169920,-0.755111,-0.179642,-1.878069,-0.860861,-5.658716e+00,-2.903888e+00,-7.886193e+00,-10.382247,-6.466902,-6.912359,-2.505351,-2.074150,-7.589756,1.0,0.000000,0.000000,0.000000,0.000000
25%,12222.000000,2011.000000,-0.029500,-0.125000,-0.018901,-0.008423,-0.002639,-0.007882,-0.002448,0.012358,0.000492,1.600148e+05,20.000000,1335.000000,3557.500000,3655.000000,1.014605e+05,4.464925e+04,1.442945e+08,3092.540972,1.410713e+07,691.000000,4.808350e+04,-6.720915e-01,-7.445491e-01,-7.413134e-01,0.848199,0.834163,0.853395,0.972661,1.003299,0.630417,1.038620,0.255723,0.197451,-5.443849e-01,-5.271013e-01,-5.159551e-01,-0.184745,-0.097444,-0.254043,0.145258,-0.18660